In [31]:
from langgraph.graph import START,END,StateGraph
from langgraph.graph.message import add_messages 
from typing import TypedDict,Annotated
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
import webbrowser 
from typing_extensions import Annotated,TypedDict
from langgraph.prebuilt import ToolNode,tools_condition
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage,BaseMessage
from googleapiclient.discovery import build
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

load_dotenv()
llm = ChatOpenAI()

@tool
def open_url(link:str):
  """a simple url opener tool that opens the url in browser"""
  webbrowser.open(link)




youtube = build("youtube", "v3", developerKey=YOUTUBE_API_KEY)





@tool
def youtube_api_search(query: str):
    """Search YouTube using official API and play the first result."""
    request = youtube.search().list(
        q=query,
        part="snippet",
        type="video",
        maxResults=1
    )
    response = request.execute()
    video_id = response["items"][0]["id"]["videoId"]
    url = f"https://www.youtube.com/watch?v={video_id}"
    webbrowser.open(url)
    return f"Playing: {response['items'][0]['snippet']['title']}"

@tool
def search_playlist(query: str) -> str:
    """Search YouTube and return the first playlist ID for a query."""
    youtube = build("youtube", "v3", developerKey=YOUTUBE_API_KEY)
    request = youtube.search().list(
        part="snippet",
        q=query,
        type="playlist",
        maxResults=1
    )
    response = request.execute()
    if response["items"]:
        return response["items"][0]["id"]["playlistId"]
    return None

@tool
def play_playlist(query: str) -> str:
    """Search and play a playlist from YouTube based on natural language query."""
    playlist_id = search_playlist(query)
    if playlist_id:
        url = f"https://www.youtube.com/playlist?list={playlist_id}"
        webbrowser.open(url)
        return f"Playing playlist for: {query}"
    else:
        return f"No playlist found for: {query}"
tools=[open_url,youtube_api_search,search_playlist,play_playlist]
tool_node=ToolNode(tools)
llm_with_tools = llm.bind_tools(tools)


class State(TypedDict):
  messages:Annotated[list[BaseMessage],add_messages]

def chat_node(state:State):
  """LLM node that may answer or request a tool call."""
  messages = state['messages']
  response = llm_with_tools.invoke(messages)
  return {"messages":[response]}

# class Link(TypedDict):
#   """Get the link for website."""

#   link:Annotated[str,...,"link for the website"]

# structured_llm = llm.with_structured_output(Link)



graph_builder = StateGraph(State)
graph_builder.add_node("chat_node",chat_node)
graph_builder.add_node("tools",tool_node)
graph_builder.add_edge(START,"chat_node")
graph_builder.add_conditional_edges("chat_node",tools_condition)
graph_builder.add_edge('tools','chat_node')
chatbot = graph_builder.compile()

CONFIG={
  "configurable":{"thread_id":"thread1"}
}


In [30]:
user_input="search for my favsong playlist and play it"
chatbot.invoke({"messages": [HumanMessage(content=user_input)]},config=CONFIG)


/var/folders/9r/pg_gv8jd0yj_gzqlzl5t2n040000gn/T/ipykernel_50713/2628191936.py:69: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  playlist_id = search_playlist(query)


{'messages': [HumanMessage(content='search for my favsong playlist and play it', additional_kwargs={}, response_metadata={}, id='bf62731c-44b3-4760-96bf-910e6ed291e7'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4OWVaRtGeiDd9UJk93K5SVlv', 'function': {'arguments': '{"query":"my favorite songs"}', 'name': 'search_playlist'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 141, 'total_tokens': 157, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C9Wr38QgLobECn1XOHa8tuGWnBoWx', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2da6f3e5-7347-44ae-a92d-80cccab70dfd-0', tool_calls=[{'name': 'search_playlist', 'args': {'query': 